In [2]:
import cPickle


pos_vects = cPickle.load(open('raw_data/pos_vects.pkl', 'rb'))
neg_vects = cPickle.load(open('raw_data/neg_vects.pkl', 'rb'))
neu_vects = cPickle.load(open('raw_data/neu_vects.pkl', 'rb'))
pos_conf = cPickle.load(open('raw_data/pos_conf.pkl', 'rb'))
neg_conf = cPickle.load(open('raw_data/neg_conf.pkl','rb'))
neu_conf = cPickle.load(open('raw_data/neu_conf.pkl','rb'))

X_vects = pos_vects + neg_vects + neu_vects
conf = pos_conf + neg_conf + neu_conf
labels = ['Positive'] * len(pos_vects) + ['Negative'] * len(neg_vects) + ['Neutral'] * len(neu_vects)

In [3]:
print len(labels)
print len(X_vects)
print len(conf)

#b_set = set(tuple(x) for x in training_vects_x)
#training_vects_x = [ list(x) for x in b_set ]
#print len(training_vects_x)

1288
1288
1288


In [4]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, to_tree, distance
import numpy as np

In [39]:
# generate the linkage matrix
X = np.array(X_vects, dtype = np.uint8)
d = distance.pdist(X,'euclidean')
#print len(X)
print len(d)
print type(d)
print d.ndim
import numpy as np

#from scipy.spatial.distance import pdist, squareform
#ds = squareform( d )
#q = lambda i,j,n: n*j - j*(j+1)/2 + i - 1 - j
#for i in xrange( 1, len(X) ):
#    for j in xrange( i ):
#        assert ds[ i, j ] == d[ q( i, j, len(X) ) ]

828828
<type 'numpy.ndarray'>
1


In [40]:
from math import sqrt

def compute_dist(x,y):
    #print sum(x), sum(y)
    sub = [a-b for a,b in zip(x,y)]
    #print sum(sub)
    times = [a*b for a,b in zip(sub,sub)]
    #print sum(times)
    return sqrt(float(sum(times)))
Z = linkage(d, method='average')
print len(X)

#for i in xrange(len(X)-1):
#    if Z[i][0] < 961 and Z[i][1] < 961:
#        print i
#        #print Z[i][0], Z[i][1]
#        print Z[i][2]
#        print compute_dist(X[Z[i][0]],X[Z[i][1]])
#        print '********************************************'
#        #assert compute_dist(X[Z[i][0]],X[Z[i][1]]) == Z[i][2]
#    #print Z[i][0],Z[i][1],Z[i][2],Z[i][3]
#print type(Z)

1288


In [41]:
r = to_tree(Z,rd=False)
print type(r)
print r.get_count(), r.get_id(), r.get_left(), r.get_right()
#for node in d:
#    print node.is_leaf(), node.count, node.id, node.dist, node.left, node.right
#    print '**********************************************************************'

<type 'instance'>
1288 2574 <scipy.cluster.hierarchy.ClusterNode instance at 0x10cf07320> <scipy.cluster.hierarchy.ClusterNode instance at 0x1143ad830>


In [42]:
class denali_tree:
    def __init__(self):
        #self.root = root
        self.vertices = []
        self.edges = []
    def inorder_walk(self, node, parent_id=None):
        if node is not None:
            self.inorder_walk(node.left, node.id)
            self.vertices.append((node.id,node.dist))
            if parent_id is not None:
                self.edges.append((parent_id, node.id))
            self.inorder_walk(node.right, node.id)
    
tree = denali_tree()
tree.inorder_walk(r)
print d.min()
print d.max()

1.0
7.14142842854


In [43]:
#print tree.vertices
#print tree.edges
from math import pow
fp = open('ClusterTree/euclidean_cluster_tree.tree','wb')
print len(tree.vertices)
fp.write('{}'.format(len(tree.vertices)))
fp.write('\n')
tree.vertices = sorted(tree.vertices, key=lambda x: x[0])
for vertex in tree.vertices:
    fp.write('{}'.format(vertex[0]))
    fp.write('\t')
    if vertex[0] < len(X):
        #fp.write('{}'.format((d.max() - vertex[1])*pow(sum(X[vertex[0]]),1.0/6)))
        fp.write('{}'.format(vertex[1]))
    else:
        #fp.write('{}'.format((d.max() - vertex[1])*1))
        fp.write('{}'.format(vertex[1]))
    fp.write('\n')
id = len(tree.vertices)
for edge in tree.edges:
    fp.write('{}'.format(edge[0]))
    fp.write('\t')
    fp.write('{}'.format(edge[1]))
    #fp.write('\t')
    #weight = abs(tree.vertices[edge[0]][1] - tree.vertices[edge[1]][1])
    #scalar_v = 1
    #fp.write('{}'.format(id))
    #fp.write('\t')
    #fp.write('{}'.format(scalar_v))
    fp.write('\n')
    #id += 1
fp.close()

2575
